In [ ]:
from nbtemplate import display_header, get_path, display_codetoggle
display_header('OptimizeGratPos.ipynb')

## Purpose
In the Guenther & Heilmann JATIS paper (2019) we floated the idea that in principle the grating placement is biased in our current scheme, which makes the centerpoint of each grating tangents to the Rowland torus. For some grating positions that means that all but this one point is "above" the Rowland torus, while gratings at other positions "cut through" the torus and might be on average "on" the Rowland torus. In the earlier case, the grating will diffract most rays above the Rowland torus and thus they will be detected a little too far out. The spectral resolution could thus be improved by moving this grating inwards just a little bit, or equivalently, the room for misalignments could be larger.

In this notebook we quantify how much the facets should be shifted compared to a layout where they are tangent to the Rowland torus.

In [ ]:
import numpy as np
import astropy.units as u
from astropy import table

from marxslynx.simulations import run_monoenergetic_simulation
from marxslynx import lynx
from marxslynx.ralfgrating import facet_table

In [ ]:
from matplotlib import pyplot as plt

%matplotlib inline

In [ ]:
wave = 22.2 * u.nm 
energy = wave.to(u.keV, equivalencies=u.spectral())

In [ ]:
instrum = {'5050': lynx.PerfectLynx(conf=lynx.conf)}

In [ ]:
facettab = facet_table(instrum['5050'].elements[2])

For every grating, we check the distance from the Rowland torus for a grid of points. The default configuration uses only the center of the gratings and makes those tangent to the Rowland torus (distance = 0). Here, we try more points, e.g. a 3 * 3 grid and then calculate the average distance. (In theory, we want to integrate over the entire surface of a grating, but computationally it's a lot easier and accurate enough to just do this for a few regularly spaced points). This shows us how we would have to move the grating to minimize the average distance form the Rowland torus.

In [ ]:
def optimize_z(gas, grating, n=3):
    '''This could replace the ``determine_xpos`` method in MeshGrid
    but for now, I'll just experiement with it here.
    Since the runtime for a full set of gratings is long, it might
    be more efficient to run it one for a given setup and tabulate
    grid positions. On the other hand, it might turn out that this is
    a small effect that does not matter at this stage.'''
    step = 2 / n
    grid = np.arange(-1 + step/2, 1., step)
    deltax = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            pos = grating.geometry['center'] + grid[i] * grating.geometry['v_y'] + grid[j] * grating.geometry['v_z']
            deltax[i, j] = gas.rowland.solve_quartic(y=pos[1], z=pos[2], interval=gas.x_range) - pos[0]
    return deltax

In [ ]:
gas = instrum['5050'].elements[2]
deltax = np.array([optimize_z(gas, e) for e in gas.elements])

In [ ]:
plt.scatter(facettab['facet_y'], facettab['facet_z'], c=np.mean(deltax, axis=(1,2)))
plt.colorbar()
plt.xlabel('Dispersion direction (mm)')
plt.ylabel('Cross-dispersion direction (mm)')

The plot above shows that all gratings should be moved slightly downwards by about 0.01 mm. However, if all gratings are moved, that's just like having a slightly larger radius for the Rowland torus. So, we look at the difference between those values and that is only of order 0.001 mm - much less than the alignment tolerance. So, we don't have to bother with this detail any longer. Uups! We could have saved ourselves the time for this investigation with a simple estimate on how far the furthest edges of a grating are on average (in mm):

In [ ]:
np.sqrt(10000**2+25**2) - 10000

This means that the *average* distance needs to be less than that, which is fully consistent with the plot above. So, all done? Yeah, for this. However, in the notebook on subaperturing, we see systematic shifts in the peak of the diffracted photons that are of order 0.05 mm, which is enough to matter. If it's not this effect, we still need to understand what it is. Even if it's something else, shifting the gratings up and down a little can potentially still help to correct for it.